In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
from jinja2 import Environment, FileSystemLoader

jinja2_env = Environment(loader=FileSystemLoader([os.getcwd()]))

In [3]:
NOT_AVAIL = [
    ("NKX2-1", "COAD"),
    ("TERT", "KICH"),
    ("TERT", "KIRP"),
]

In [4]:
# just for get disease and gene information
%time adf = pd.read_csv('../__results/aca_pval_with_sc_info.csv')

CPU times: user 68.1 ms, sys: 11.2 ms, total: 79.3 ms
Wall time: 79.6 ms


In [5]:
adf.head(2)

,disease,gene_name,mkid,mclv,strand,aclv,N_on,T_on,N_off,T_off,...,N_on_ratio,T_on_ratio,N2T_ratio_diff,diff_is_significant,N2T_ratio_change,N2T_ratio_change_sig,has_diff_aca,sc_list,src_list,num_sc
0,BLCA,ABL1,chr9|ABL1|+|133589972,133589972,+,133763062,1,23,18,382,...,0.052632,0.05679,0.004159,False,up,NaN,False,133761070,protein_coding,1
1,BLCA,ABL1,chr9|ABL1|+|133763062,133763062,+,133763062,19,405,0,0,...,1.000000,1.00000,0.000000,False,down,NaN,False,133761070,protein_coding,1


In [6]:
reported_gd_pairs = adf.query('has_diff_aca == True')[['gene_name', 'disease']].drop_duplicates().values
reported_gd_pairs = [tuple(_) for _ in reported_gd_pairs]

In [7]:
genes = np.sort(adf.gene_name.unique()).tolist()
dises = np.sort(adf.disease.unique()).tolist()

In [8]:
assert len(genes) == 114
assert len(dises) == 14

If htmlmin isn't installed, try `!pip install htmlmin`

In [9]:
import htmlmin

In [10]:
template = jinja2_env.get_template('templates/index.jinja2')
with open('../__results/all-apa-cases.html', 'wt') as opf:
    content = template.render(
        diseases=dises,
        genes=genes,
        reported_gd_pairs=reported_gd_pairs, 
        na_list=NOT_AVAIL
    )
    # minify css and js
    content = ' '.join(content.split())
    content = htmlmin.minify(content, remove_empty_space=True)
    opf.write(content)

In [11]:
# Used to generat markdown, deprecated; prefer html

# def wrap(str_):
#     return '|' + str_ + '|'

# headers = wrap('|'.join([' '] + dises))

# seps = wrap('|'.join(['---'] * (len(dises) + 1)))

# url_temp = '[✓](http://bcgsc.ca/downloads/zxue/all_diff_apa_114_genes_14_diseases/{gene_name}_{disease}.png)'

# with open('./diff_apa_table.md', 'wt') as opf:
#     opf.write('{0}\n'.format(headers))
#     opf.write('{0}\n'.format(seps))
#     for g in genes:
#         opf.write('|' + g + '|')
#         for d in dises:
#             opf.write(url_temp.format(gene_name=g, disease=d) + '|') 
#         opf.write('\n')